In [2]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_30.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "30_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "30_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "30_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "30_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "30_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "30_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 30_sub_mat_1.csv has been saved.
Submatrix 30_sub_mat_2.csv has been saved.
Submatrix 30_sub_mat_3.csv has been saved.
submatrix 30_cross_matrix_1_2.csv has been saved
submatrix 30_cross_matrix_1_3.csv has been saved
submatrix 30_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.886723,0.888853,0.881355,0.863044,0.745869,0.818807,0.799246,0.828254,0.678789,0.828728,...,0.621765,0.86247,0.812492,0.839641,0.806562,0.812992,0.889527,0.796245,0.703957,0.828322
21085.0,0.848789,0.843727,0.844228,0.767752,0.761175,0.799058,-0.763502,0.814996,0.712428,-0.812648,...,0.559698,-0.871367,-0.793868,0.820129,-0.799417,0.754529,0.866877,-0.815722,0.686074,0.801855
21087.0,0.896876,0.847538,0.877204,-0.802003,0.780361,-0.85461,-0.876213,0.877332,0.785455,-0.866239,...,0.615909,-0.901571,-0.802729,0.777313,-0.820954,-0.743806,-0.913162,-0.866444,0.70802,0.803534
21089.0,0.88202,0.829053,0.876665,0.846573,0.749223,0.806076,0.820872,0.826002,0.709906,0.79783,...,0.596695,0.857119,0.714237,0.764578,0.780891,0.756971,0.8921,0.807487,0.689587,0.75208
21091.0,0.81804,0.72986,0.836899,0.788456,0.851786,0.756755,0.77873,0.778449,0.846424,0.725844,...,-0.461149,0.729933,0.558384,0.674332,-0.57909,-0.777467,0.765127,-0.697976,0.713105,0.784644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.813609,0.9583,-0.73812,0.840264,-0.751175,-0.857177,-0.800658,-0.856563,0.691827,-0.882555,...,-0.68813,-0.912759,-0.941564,-0.955752,-0.75413,-0.853141,-0.828385,-0.835323,-0.59962,0.804416
38091.0,0.771471,0.805046,-0.754653,-0.755946,-0.742461,-0.825445,-0.777744,-0.823355,0.708734,-0.835902,...,-0.605752,-0.824709,-0.76772,-0.779846,-0.719149,-0.790634,-0.850457,-0.795362,-0.672822,-0.696512
38093.0,0.944925,0.916928,-0.91756,0.926927,-0.91216,-0.939892,-0.947073,0.934256,0.925938,-0.930287,...,-0.59321,-0.937413,-0.828585,-0.88745,-0.806376,-0.899992,-0.959914,-0.878407,0.764824,0.862319
38095.0,0.831504,0.834016,0.831677,0.916182,0.819423,-0.84273,-0.874164,0.832492,0.779498,-0.867389,...,-0.461065,-0.865439,-0.75304,-0.77064,-0.69335,-0.836069,-0.847688,-0.803716,0.741211,0.740729
